In [1]:
import os
import cntk as ct

from src.ferplus import FERPlusReader, FERPlusParameters
from src.models import *

In [2]:
def cost_func(training_mode, prediction, target):
    '''
    We use cross entropy in most mode, except for the multi-label mode, which require treating
    multiple labels exactly the same.
    '''
    train_loss = None
    if training_mode == 'majority' or training_mode == 'probability' or training_mode == 'crossentropy': 
        # Cross Entropy.
        train_loss = ct.negate(ct.reduce_sum(ct.element_times(target, ct.log(prediction)), axis=-1))
    elif training_mode == 'multi_target':
        train_loss = ct.negate(ct.log(ct.reduce_max(ct.element_times(target, prediction), axis=-1)))

    return train_loss

In [3]:
def binarizar_array(array):
    max_index = np.argmax(array)  # Encontra o índice do valor máximo
    binarizado = np.zeros_like(array)  # Cria um array com zeros do mesmo tamanho que o array de entrada
    binarizado[max_index] = 1  # Define o valor 1 na posição do valor máximo
    return binarizado

In [4]:
# parametros
model_name='VGG13'

training_mode = "majority"
base_folder = 'data'

test_folders  = ['FER2013Test']

In [5]:
# folders
output_model_path   = os.path.join(base_folder, R'models')
output_model_folder = os.path.join(output_model_path, model_name + '_' + training_mode)

if not os.path.exists(output_model_folder):
    os.makedirs(output_model_folder)

In [6]:
emotion_table = {'neutral'  : 0, 
                 'happiness': 1, 
                 'surprise' : 2, 
                 'sadness'  : 3, 
                 'anger'    : 4, 
                 'disgust'  : 5, 
                 'fear'     : 6, 
                 'contempt' : 7}

num_classes = len(emotion_table)

In [7]:
# leitura do modelo
model = build_model(num_classes, model_name)

In [8]:
# set the input variables.
input_var = ct.input((1, model.input_height, model.input_width), np.float32)
label_var = ct.input((num_classes), np.float32)

In [9]:
# training_mode interfere nos labels, fazendo com que seja 0 e 1 ou entre 0 e 1.
# por algum motivo, no test_and_val_params ele está estatico em 'majatory'
test_and_val_params = FERPlusParameters(num_classes, model.input_height, model.input_width, training_mode, True)
# test_and_val_params = FERPlusParameters(num_classes, model.input_height, model.input_width, "majority", True)
test_data_reader = FERPlusReader.create(base_folder, test_folders, "label.csv", test_and_val_params)

In [10]:
epoch_size = test_data_reader.size()
minibatch_size = 32

In [11]:
# get the probalistic output of the model.
z    = model.model(input_var)
pred = ct.softmax(z)

In [12]:
# Training config
lr_per_minibatch       = [model.learning_rate]*20 + [model.learning_rate / 2.0]*20 + [model.learning_rate / 10.0]
mm_time_constant       = -minibatch_size/np.log(0.9)
lr_schedule            = ct.learning_rate_schedule(lr_per_minibatch, unit=ct.UnitType.minibatch, epoch_size=epoch_size)
mm_schedule            = ct.momentum_as_time_constant_schedule(mm_time_constant)

# loss and error cost
train_loss = cost_func(training_mode, pred, label_var)
pe         = ct.classification_error(z, label_var)

In [13]:
# construct the trainer
learner = ct.momentum_sgd(z.parameters, lr_schedule, mm_schedule)
trainer = ct.Trainer(z, (train_loss, pe), learner)
trainer.total_number_of_samples_seen

0

In [14]:
best_epoch = 80
print(os.path.join(output_model_folder, "model_{}".format(best_epoch)))
trainer.restore_from_checkpoint(os.path.join(output_model_folder, "model_{}".format(best_epoch)))
trainer.total_number_of_samples_seen

data/models/VGG13_majority/model_80


2028645

In [15]:
true_labels_batch = []
y_predicts_batch = []

while test_data_reader.has_more():
    images, labels, current_batch_size = test_data_reader.next_minibatch(minibatch_size)
    true_labels_batch.append(labels)
    y_predicts_batch.append(trainer.model.eval({input_var: images}))

In [16]:
true_labels = np.concatenate(true_labels_batch, axis=0)
true_labels.shape

(3137, 8)

In [17]:
y_predicts = np.concatenate(y_predicts_batch, axis=0)
y_predicts.shape

(3137, 8)

In [18]:
true_labels

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.]], dtype=float32)

In [19]:
y_predicts

array([[  5.18589878,  -0.47918698,  -4.85144663, ...,   0.75687623,
         -4.6952939 ,   2.55302906],
       [  5.80026007,   0.03826645,  -5.3946991 , ...,  -1.62808061,
         -4.59390116,  -0.2849265 ],
       [ -2.16070414,   9.9881382 ,   3.40021968, ...,  -3.99011946,
         -0.90389687,  -3.14976597],
       ..., 
       [  7.6844635 ,   7.68678284,  -2.67586732, ...,  -4.2151227 ,
         -5.42998028,   3.21308613],
       [ -4.86283588,  18.04737282,   1.03866386, ...,  -4.47006512,
         -3.56063461,  -3.897048  ],
       [ -3.79545736,   8.92692375,   4.41546392, ...,  -3.92081785,
          2.10619664,  -2.31982589]], dtype=float32)

In [20]:
y_predicts_bin = np.array([ binarizar_array(arr) for arr in y_predicts ])
y_predicts_bin

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.]], dtype=float32)